In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import re

In [2]:
def transform_currentPrice_text(input_currentPriceLine):
    current_price_rep = input_currentPriceLine.replace('현재가 ', '')
    current_price_rep = current_price_rep.replace('상승 ', '+')
    current_price_rep = current_price_rep.replace('플러스 ', '+')
    current_price_rep = current_price_rep.replace('하락 ', '-')
    current_price_rep = current_price_rep.replace('마이너스 ', '-')
    current_price_rep = current_price_rep.replace(' 퍼센트', '%')
    current_price_rep = current_price_rep.replace('전일대비', '')
    current_price_rep = current_price_rep.replace('  ', ' ')
    return current_price_rep

In [3]:
def getCodeList():
    code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0] 
    # 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줌 
    code_df.종목코드 = code_df.종목코드.map('{:06d}'.format) 
    # 우리가 필요한 것은 회사명과 종목코드이기 때문에 필요없는 column들은 제외해준다.
    code_df = code_df[['회사명', '종목코드']] 
    # 한글로된 컬럼명을 영어로 바꿔준다. 
    code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'}) 
    return code_df


In [4]:
def getStockPage(code):
    test_url = 'https://finance.naver.com/item/main.nhn?code={code}'.format(code=code)
    test_result = requests.get(test_url)
    soup = BeautifulSoup(test_result.content, "html.parser")
    print(test_url)
    return soup

In [5]:
def getCode(stock_name):
    code = code_df.query("name=='{}'".format(stock_name))['code'].to_string(index=False) 
    code = code[1:7]
    return code

In [6]:
def getBasicInfo(soup, item_name, code):
    pbr = 0
    per = 0
    sameBusinessPER = 0
    stock_type = ''
    
    for x in soup.find_all('em',id='_pbr'):
        if not (x.text.strip()):
            pbr = 0
        else: pbr = x.text

    for x in soup.find_all('em',id='_per'):
        per = x.text

#         similar per 
    trTag = soup.find_all("tr", {"class": "strong"}) # 태그 p, 속성값 no_today 찾기
    for i in trTag:
        if("동일업종" in i.text):
            sameBusinessPER = i.text
            sameBusinessPER = sameBusinessPER.replace("동일업종 PER","").strip()

    for tag in soup.find_all(re.compile("dd")):
        str = tag.get_text()
        if("현재가" in str):
            current_price = str
        elif("전일가" in str):
            yesterday_price = str.replace("전일가", "").strip()
        elif("시가" in str):
            day_start_price = str.replace("시가", "").strip()   
        elif("고가" in str):
            day_high_price = str.replace("고가", "").strip()
        elif("저가" in str):
            day_low_price = str.replace("저가", "").strip()   
        elif("거래량" in str):
            day_trading_vol = str.replace("거래량", "").strip()
        elif("거래대금" in str):
            day_trading_price = str.replace("거래대금", "").strip()   
            if("백만" in str):
                day_trading_price = day_trading_price.replace("백만", ",000,000").strip()
        elif("코스닥" in str):
            stock_type = '코스닥'
        elif("코스피" in str):
            stock_type = '코스피'
                 
    current_price_rep = transform_currentPrice_text(current_price)
    
    str_cnt = 0
    temp_str = ''
    dict = {"종목명": item_name, "종목코드": code, "현재가":"","전일대비금액":"", "전일대비%":"","전일가":"","시가":"","고가":"","저가":"",
           "거래량":"","거래대금":"","PBR":"","PER":"","BIZ_PER":"", "TYPE":""}

    for i in current_price_rep:
        temp_str += i
        if(i == ' ' or i == '%'):
            if(str_cnt == 0):
                dict['현재가'] = temp_str
            elif(str_cnt == 1):
                dict['전일대비금액'] = temp_str
            elif(str_cnt == 2):
                dict['전일대비%'] = temp_str
            str_cnt += 1
            temp_str = ''

    for item_name in dict:
        if(item_name == '전일가'):
            dict[item_name] = yesterday_price
        elif(item_name == '시가'):
            dict[item_name] = day_start_price
        elif(item_name == '고가'):
            dict[item_name] = day_high_price
        elif(item_name == '저가'):
            dict[item_name] = day_low_price
        elif(item_name == '거래량'):
            dict[item_name] = day_trading_vol
        elif(item_name == '거래대금'):
            dict[item_name] = day_trading_price
        elif(item_name == 'PBR'):
            dict[item_name] = pbr
        elif(item_name == 'PER'):
            dict[item_name] = per      
        elif(item_name == 'BIZ_PER'):
            dict[item_name] = sameBusinessPER
        elif(item_name == 'TYPE'):
            dict[item_name] = stock_type      
    return dict

In [7]:
current_price = 0
yesterday_price = 0
day_start_price = 0
day_high_price = 0
day_low_price = 0
day_trading_vol = 0
day_trading_price = 0

stockList = []


In [ ]:
# item_name_list = ["모두투어리츠"]
item_name_list = []
cnt_item = 0
cnt_show = 0

for i in getCodeList().name:
    item_name_list.append(i)
    cnt_item += 1  


for item_name in item_name_list[1000:2000]:
    code_df = getCodeList()
    code = getCode(item_name)
    soup = getStockPage(code)
    basicInfo = getBasicInfo(soup, item_name, code)
    stockList.append(basicInfo)
    cnt_show += 1
    print(item_name)
    print(cnt_show)



In [ ]:
stock_df = pd.DataFrame(stockList)

In [ ]:
# stock_df.to_excel("output2.xlsx")
stock_df.columns
print(stock_df.columns)
print(stock_df.values)

In [ ]:
stock_df = stock_df.loc[:,['종목코드','종목명','TYPE', '거래량', '거래대금', 
'전일대비금액','전일대비%','전일가','현재가','시가','고가','저가','PBR', 'PER', 'BIZ_PER']]
stock_df_rows = stock_df.values.tolist()

In [ ]:
import cx_Oracle
con = cx_Oracle.connect('daechul/sysdba@127.0.0.1/orcl')
cur = con.cursor()


In [ ]:
# cur.bindarraysize = 10
# cur.setinputsizes(int, 100)
start_val = 0
for i in range(1000):
    end_val = start_val + 1
    cur.executemany("insert into STOCK_OVERVIEW values (:1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11, :12, :13, :14, :15, SYSDATE)", stock_df_rows[start_val:end_val])
    con.commit()
    start_val += 1


In [ ]:
# # cur.bindarraysize = 100
# # cur.setinputsizes(int, 100)
# stock_values = stock_df.values
# # print(rows)
# for i in stock_values[9:10]:
# #     print(i)
#     i = i.tolist()
#     print(i[12])
#     print(i)
#     print(type(i))
#     cur.executemany("insert into STOCK_TEST values (:1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11, :12, :13, :14, :15)", i)
#     con.commit()
# # cur.executemany("insert into STOCK_TEST values (:1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11, :12, :13, :14, :15)", z)
# # con.commit()

In [ ]:
cur.close()
con.close()